In [1]:
# Import libraries
import os
import sys
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from beam_corot.ComplBeam import ComplBeam
from beam_corot.CoRot import CoRot
from utils import save_load, c2_to_node

C:\Users\julio\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
C:\Users\julio\AppData\Local\Temp\ipykernel_27032\4282684689.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [7]:
# Model input json file name
f_model_json = "straight_beam_e_offset_static.json"

# Input files folder
inputfolder = os.path.join(os.getcwd(),'straight_beam')
mainfile = os.path.join(inputfolder,f_model_json)

# - Instantiate beam
save_load([0], inputfolder, onlyy=True) # Creates a force file
beam = ComplBeam(mainfile)

# - Get the radius location of nodes
r = beam.nodeLocations[:,2] # z-axis position
beam.nodeLocations

pos = np.concatenate((beam.nodeLocations,np.zeros((2,3))), axis=1)
print(f'{pos = }')

-----------
ComplBeam Model Created
Static analysis done
pos = array([[0.1, 0. , 0. , 0. , 0. , 0. ],
       [0.1, 0. , 1. , 0. , 0. , 0. ]])


array([[1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.]])

In [9]:
# Nodes are located in the elastic centre
c2_pos = beam.c2Input[:, 1:4]
node_pos = beam.nodeLocations # Elastic centre position in the spline. Nx2
r_node2c2 = c2_pos - node_pos
r_node2c2

array([[-0.1,  0. ,  0. ],
       [-0.1,  0. ,  0. ]])

In [11]:
# Moment only in z-axis. It is not affected by the offset
loads = np.zeros((pos.shape[0],6))
loads[:,5] = 1

c2_to_node(beam, loads)

array([[0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.]])

In [12]:
# Force in x-axis. It does not induce a moment since the offset is in the x-axis
loads = np.zeros((pos.shape[0],6))
loads[:,0] = 1

c2_to_node(beam, loads)

array([[1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.]])

In [13]:
# Force in y-axis. It creates moment in z-axis
loads = np.zeros((pos.shape[0],6))
loads[:,1] = 1

c2_to_node(beam, loads)

array([[ 0. ,  1. ,  0. ,  0. ,  0. , -0.2],
       [ 0. ,  1. ,  0. ,  0. ,  0. , -0.2]])

In [14]:
# Force in z-axis. It creates moment in y-axis
loads = np.zeros((pos.shape[0],6))
loads[:,2] = 1

c2_to_node(beam, loads)


array([[0. , 0. , 1. , 0. , 0.2, 0. ],
       [0. , 0. , 1. , 0. , 0.2, 0. ]])